## Запуск Spark

In [ ]:
import sys
import os

spark_home='/opt/spark-1.5.1-bin-hadoop2.6/'
# addin pyspark to current path
sys.path.append( spark_home+'/python' )
sys.path.append( spark_home+'/python/lib/py4j-0.8.2.1-src.zip' )

# Установка локальных переменных
os.environ["SPARK_HOME"] = spark_home
os.environ["HADOOP_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop'
os.environ["HADOOP_YARN_HOME"] = '/opt/cloudera/parcels/CDH/lib/hadoop-yarn'
os.environ["YARN_CONF_DIR"] = '/etc/hadoop/conf.cloudera.yarn'
os.environ["SPARK_CLASSPATH"] = '/etc/hive/conf.cloudera.hive1'
os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local[4] pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
conf = SparkConf ().set( 'spark.app.name', 'test');
sc = SparkContext (conf= conf)

# Фичи

<img src='images/dense_vector.PNG'>

### Обычный "плотный" вектор

в MLlib можно задавать его
- через список 
  [-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46]

In [ ]:
data = sc.parallelize([[-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46],
  [-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46]])

In [ ]:
data.take(2)

- через numpy.array (что более эффективно) np.array([-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46])


In [ ]:
import numpy as np
a = np.array([-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46])

- класс "вектор" Vectors.dense ([-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46])

In [ ]:
from pyspark.mllib.linalg import Vectors
v = Vectors.dense ([-2.91, 4.08, 0, 0, 0, 0, 2.48, 0, 0, 1.46])

### <br><br><br><br> Разреженный вектор

Задавать разреженные вектора можно при помощи 
- SciPy csc_matrix с одной колонкой<br>
Пример: sv2 = sps.csc_matrix((np.array([1.0, 10.0]), np.array([0, 1, 6, 9]), np.array([-2.91, 4.08, 2.48, 1.46])), shape = (10, 1))

In [ ]:
# Use a single-column SciPy csc_matrix as a sparse vector.
import scipy.sparse as sps

- MLibs SparseVector<br>
Пример: Vectors.sparse(10, [0,1,6,9], [-2.91, 4.08, 2.48, 1.46])

In [ ]:
from pyspark.mllib.linalg import Vectors
a = Vectors.sparse(10, [0,1,6,9], [-2.91, 4.08, 2.48, 1.46])

In [ ]:
a

<br><br><br><br><br><br>
<h2>Размеченные точки</h2>

<img src='images/labeled_point.PNG'>

Класс LabeledPoint

In [ ]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint (1, a)

**LabeledPoint.features** - любой из описанных выше векторов

**LabeledPoint.label** - float<br>
Может быть:
- 0.0, 1.0 для бинарной классификации
- 0.0, 1.0, 2.0, ... для мультиклассовой классификации
- любое число для регрессии    

<br><br><br><br><br><br>
<h2>Загрузка данных</h2>

Можно создать датасет LabeledPoints самостоятельно<br>
- Файл с примерами <a href='http://5.9.102.237:8888/filebrowser/view//testdata/kaggle/train.csv'>/testdata/kaggle/train.csv</a>

In [ ]:
data = sc.textFile ("/testdata/kaggle/train.csv")

In [ ]:
data
.map(lambda x: [float(v) for v in x.split("\t")])\
.map (lambda x: LabeledPoint (x[0], x[1:])).take(5)

## <br><br><br><br><br><br><br><br> Загрузка Lib SVM файлов

У lib svm есть свой формат хранения разреженных фич<br>
Пример: <a href='http://5.9.102.237:8888/filebrowser/view//testdata/mllib_demo/libsvm_dataset/part-00000'>/testdata/mllib_demo/libsvm_dataset</a><br>
Команда MLUtils.loadLibSVMFile(sc,path)

In [ ]:
! hadoop fs -cat /testdata/mllib_demo/libsvm_dataset/part-00000 | head

In [ ]:
from pyspark.mllib.util import MLUtils
svm_result = MLUtils.loadLibSVMFile(sc, "/testdata/mllib_demo/libsvm_dataset")

In [ ]:
svm_result.take(5)

## Остановка Spark

In [ ]:
sc.stop()